In [1]:
import pandas as pd
import random
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import optuna   
import json
import os
from optuna.visualization import plot_optimization_history

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"

In [3]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

Score Trend Changes Score

In [4]:
def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(y_df["is_changed_trend_test"][:-1], y_df["is_changed_trend_predict"][:-1], digits=4)

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

Carga de datos

In [5]:
# Cargar datos
train = pd.read_csv("../../../data/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/validation_set.csv", parse_dates=["date"])
test_set = pd.read_csv("../../../data/test_set.csv", parse_dates=['date'])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values
X_test = test_set.drop(columns=['target_trend','date']).values
y_test = test_set['target_trend'].values

IMPORTANTE: Hay que sumar a la columna de prediccion porque -1 no funciona en funcion de LOSS

In [6]:
y_train += 1
y_val += 1

Integracion de metrica trend_changes_score en la funcion objetivo Optuna

In [7]:
def objective(trial):
    set_seeds(SEED)  # Fijar semilla antes de cada trial
    penalty = trial.suggest_categorical("penalty", ["l1", "l2", "elasticnet"])
    C = trial.suggest_float("C", 1e-3, 50, log=True)
    l1_ratio = None
    if penalty == "elasticnet":
        l1_ratio = trial.suggest_float("l1_ratio", 0.1, 0.9)

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            multi_class="multinomial",
            solver="saga",
            penalty=penalty,
            C=C,
            l1_ratio=l1_ratio,
            max_iter=800,
            random_state=SEED
        ))
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    #score = trend_changes_true(y_val, y_pred)  # <- Métrica anterior
    score = balanced_accuracy_score(y_val, y_pred) # <- Métrica actual
    return score

Ejecucion de estudio con Optuna

In [8]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=300)

[I 2025-08-21 09:52:30,909] A new study created in memory with name: no-name-10317641-c7dc-43d8-b924-3ffad6fdce88


/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-08-21 09:52:31,425] Trial 0 finished with value: 0.7714646464646465 and parameters: {'penalty': 'l2', 'C': 4.9020352232191025}. Best is trial 0 with value: 0.7714646464646465.
/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-08-21 09:52:32,069] Trial 1 finished with value: 0.752946127946128 and parameters: {'penalty': 'l1', 'C': 5.8610214330700705}. Best is trial 0 with value: 0.7714646464646465.
[I 2025-08-21 09:52:32,363] Trial 2 finished with value: 0.8013468013468014 and parameters: {'penalty': 'l1', 'C': 0.22602738844381381}. Best is trial 2 with value: 0.8013468013468014.
[I 2025-08-21 

In [9]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [10]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'penalty': 'l1', 'C': 0.23241163608599685}
Mejor score de f1-score: 0.8051


GUARDAR EN JSON

In [ ]:
# # Guardar los mejores hiperparámetros y su valor
# history = []
# if os.path.exists("best_hyperparams.json"):
#     try:
#         with open("best_hyperparams.json", "r") as f:
#             history = json.load(f)
#     except (json.JSONDecodeError, ValueError):
#         history = []

# # Guardar ambos en un solo diccionario
# history.append({
#     "params": study.best_params,
#     "value": study.best_value
# })

# with open("best_hyperparams.json", "w") as f:
#     json.dump(history, f, indent=2)

CARGAR HIPERPARAMETROS DESDE JSON

In [ ]:
# # Cargar historial de hiperparámetros y valores
# with open("best_hyperparams.json", "r") as f:
#     history = json.load(f)

# # Escoger el último (más reciente)
# best_params = history[-1]["params"]
# best_value = history[-1]["value"]

# # Si quieres ver todos:
# for i, entry in enumerate(history):
#     print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# # Si quieres escoger uno específico (por índice):
# # best_params = history[indice_que_quieras]["params"]
# # best_value = history[indice_que_quieras]["value"]

Hiperparámetros #1: {'penalty': 'l1', 'C': 0.23241163608599685}, Valor: 0.8051346801346803


In [15]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
best_params = study.best_params
final_l1_ratio = best_params["l1_ratio"] if best_params["penalty"] == "elasticnet" else None
final_model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        multi_class="multinomial",
        solver="saga",
        penalty=best_params["penalty"],
        C=best_params["C"],
        l1_ratio=final_l1_ratio,
        max_iter=800,
        random_state=SEED
    ))
])

final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)
print("Trend Change F1 Score:\n", trend_changes_score(y_val, y_pred))

Trend Change F1 Score:
               precision    recall  f1-score   support

       False     0.8761    0.8839    0.8800       112
        True     0.1875    0.1765    0.1818        17

    accuracy                         0.7907       129
   macro avg     0.5318    0.5302    0.5309       129
weighted avg     0.7854    0.7907    0.7880       129



In [17]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val, y_pred, digits=4)
print("LightGBM Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_pred))

LightGBM Report:
               precision    recall  f1-score   support

           0     0.8800    0.9167    0.8980        24
           1     0.5882    0.5556    0.5714        18
           2     0.9432    0.9432    0.9432        88

    accuracy                         0.8846       130
   macro avg     0.8038    0.8051    0.8042       130
weighted avg     0.8824    0.8846    0.8834       130

Balanced accuracy: 0.8051346801346803


In [14]:
# Obtener predicciones
y_pred = final_model.predict(X_test)
print("Trend Change F1 Score:\n", trend_changes_score(y_test, y_pred))

Trend Change F1 Score:
               precision    recall  f1-score   support

       False     0.9310    0.9076    0.9191       119
        True     0.2143    0.2727    0.2400        11

    accuracy                         0.8538       130
   macro avg     0.5727    0.5901    0.5796       130
weighted avg     0.8704    0.8538    0.8617       130

